In [28]:
import torch
import torch.distributions as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [29]:
# Creating Ising spins and Calculating Hamiltonian of the Ising system
class Ising():
    
    def __init__(self, nRow, nCol):
        self.spins = torch.zeros(nRow, nCol)
        self.probs = torch.rand(nRow, nCol)
        for i in range(nRow):
            for j in range(nCol):
                if self.probs[i,j] < 0.5:
                    self.spins[i,j] = 1
                else:
                    self.spins[i,j] = -1
    
    def Hamiltonian(self):
        H = 0.
        J = 1.
        nRow = self.spins.size()[0]
        nCol = self.spins.size()[1]
        for i in range(nRow):
            for j in range(nCol):
                if i < 1:
                    H -= J * self.spins[i,j] * self.spins[i+1,j]
                elif i > nRow - 2:
                    H -= J * self.spins[i,j] * self.spins[i-1,j]
                else:
                    H -= J * self.spins[i,j] * self.spins[i+1,j]
                    H -= J * self.spins[i,j] * self.spins[i-1,j]
                
                if j < 1:
                    H -= J * self.spins[i,j] * self.spins[i,j+1]
                elif j > nCol - 2:
                    H -= J * self.spins[i,j] * self.spins[i,j-1]
                else:
                    H -= J * self.spins[i,j] * self.spins[i,j+1]
                    H -= J * self.spins[i,j] * self.spins[i,j-1]
        return H/2   #to avoid double count

In [30]:
# Creating the RBM Architecture (weights, biases)
class RBM():
    
    def __init__(self, nv, nh):
        self.W = torch.randn(nv, nh)
        self.a = torch.randn(nh)
        self.b = torch.randn(nv)
        
    def Hamiltonian(self, v, h): 
        ah = torch.dot(self.a, h)
        bv = torch.dot(self.b, v)
        vWh = torch.dot(v, torch.mv(self.W, h))
        H = - ah - bv - vWh
        return H

In [31]:
# Set Training process
def Train(params, loss):
        optimizer = optim.SGD(params, lr=0.01)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [120]:
# Creating the training data set by using Metropolis algorithm
nData = 3
nRow = 3
nCol = 3

for i in range(nData):
    
    ising = Ising(nRow, nCol)
    H_new = ising.Hamiltonian()
    
    # Reshape of a matrix of Ising spins to a vector as the visible layer
    global training_data

    spin = ising.spins.view(nRow*nCol)
    v = (1 - spin)/2   # spin 1 --> 0 ,   spin -1 --> 1
    
    # save visible layers to the training data set
    if i == 0:
        training_data = v
    else:
        if H_new <= H:
            training_data = torch.stack((training_data, v), dim = 0)
        else:
            B_dist = dist.Bernoulli(torch.exp(H - H_new))
            B = B_dist.sample()
            if B == 1:
                training_data = torch.stack((training_data, v), dim = 0)
            else:
                training_data = torch.stack((training_data, training_data[i-1,:]), dim = 0)
    H = H_new

print(training_data)

RuntimeError: invalid argument 0: Tensors must have same number of dimensions: got 3 and 2 at /pytorch/aten/src/TH/generic/THTensor.cpp:702

In [18]:
# Specify the parameters of RBM
nv = len(v)
nh = 4
batch_size = 1

# Generate random hidden layer
rand_bernoulli = dist.Bernoulli(torch.rand(nh))
h = rand_bernoulli.sample()

In [199]:
# Traing RBM
rbm = RBM(nv, nh)
rbm.W = 
params = (rbm.W, rbm.a, rbm.b)

num_epoch = 10
target = ising.Hamiltonian()
loss_fn = nn.MSELoss()

for epoch in range(0, num_epoch + 1):
    
    output = rbm.Hamiltonian(v, h)
    loss = loss_fn(output, target)
    
    if epoch > 0:
        Train(params, loss)
    
    print('epoch {}: W = {}'.format(epoch, rbm.W))
    print('\t a = {}'.format(rbm.a))
    print('\t b = {}'.format(rbm.b))
    print('\t loss = {}'.format(loss))

epoch 0: W = tensor([[ 0.6723, -0.3035, -1.2598,  1.1631],
        [-0.9218,  0.0252, -0.4875, -0.4519],
        [ 0.1067, -0.7669,  0.6226,  1.8961],
        [ 1.3410,  0.1902, -0.2706,  1.5131]])
	 a = tensor([ 0.3554, -1.5425, -1.5649, -0.2119])
	 b = tensor([ 0.1077, -0.8345,  0.9772,  0.2664])
	 loss = 21.98969078063965


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn